# First Dot ~ all frequencies: 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns
import scipy
import joblib
import h5io
import dask.array as da 

import statsmodels
from statsmodels import stats
from statsmodels.stats import multitest

# Import required code for visualizing example models
from fooof import FOOOF
from fooof.sim.gen import gen_power_spectrum
from fooof.sim.utils import set_random_seed
from fooof.plts.spectra import plot_spectra
from fooof.plts.annotate import plot_annotated_model
from neurodsp.utils import create_times
from neurodsp.plts.time_series import plot_time_series
from neurodsp.spectral import compute_spectrum, rotate_powerlaw
from neurodsp.plts.spectral import plot_power_spectra


In [ ]:
## Prep paths ##

subject = ''
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

# load preproc functions
%run ../../scripts/preproc_functions.py

In [ ]:
## Load Neural Data

# load
first_dot_epochs = mne.read_epochs(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif")

# get good epochs (for behavioral data only)
good_epochs = [i for i,x in enumerate(first_dot_epochs.get_annotations_per_epoch()) if not x]
bad_epochs = [i for i,x in enumerate(first_dot_epochs.get_annotations_per_epoch()) if  x]

# load behavioral data
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")

# set info as metadata
first_dot_epochs.metadata = first_dot_data

# onlt good epochs
first_dot_epochs = first_dot_epochs[good_epochs]

In [ ]:
## Dictionary of electrode locations ##

# Pull mapping ROI to elecs
%run ../../scripts/roi.py
ROIs = ROIs[subject]

## prep lists

# limbic ROI
hc_list = []
hc_indices = []
hc_names = []
ofc_list = []
ofc_indices = []
ofc_names = []
amyg_list = []
amyg_names = [] 
amyg_indices = []
cing_list = []
cing_names = [] 
cing_indices = []
sgACC_list = []
sgACC_names = [] 
sgACC_indices = []
dACC_list = []
dACC_names = [] 
dACC_indices = []
sfg_list = []
sfg_names = [] 
sfg_indices = []
mfg_list = []
mfg_names = [] 
mfg_indices = []

# control ROI
insula_list = []
insula_names = []  
insula_indices = []
dlpfc_list = []
dlpfc_names = []  
dlpfc_indices = []
ec_list = []
ec_names = []  
ec_indices = []

# exclude bad ROI from list
pairs_long_name = [ch.split('-') for ch in first_dot_epochs.info['ch_names']]
bidx = len(first_dot_epochs.info['bads']) +1
pairs_name = pairs_long_name[bidx:len(pairs_long_name)]

# sort ROI into lists
for ix in range(0, len(pairs_name)):
    if pairs_name[ix][0] in ROIs['hc'] or pairs_name[ix][1] in ROIs['hc']:
        hc_list.append(first_dot_epochs.info['ch_names'][ix + bidx])
        hc_names.append(pairs_name[ix])
        hc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['ofc'] or pairs_name[ix][1] in ROIs['ofc']:
        ofc_list.append(first_dot_epochs.info['ch_names'][ix + bidx])
        ofc_names.append(pairs_name[ix])
        ofc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['amyg'] or pairs_name[ix][1] in ROIs['amyg']:
        amyg_list.append(first_dot_epochs.info['ch_names'][ix + bidx])       
        amyg_names.append(pairs_name[ix])
        amyg_indices.append(ix)
    if pairs_name[ix][0] in ROIs['cing'] or pairs_name[ix][1] in ROIs['cing']:
        cing_list.append(first_dot_epochs.info['ch_names'][ix + bidx])       
        cing_names.append(pairs_name[ix])
        cing_indices.append(ix)
    if pairs_name[ix][0] in ROIs['sgACC'] or pairs_name[ix][1] in ROIs['sgACC']:
        sgACC_list.append(first_dot_epochs.info['ch_names'][ix + bidx])       
        sgACC_names.append(pairs_name[ix])
        sgACC_indices.append(ix)
    if pairs_name[ix][0] in ROIs['dACC'] or pairs_name[ix][1] in ROIs['dACC']:
        dACC_list.append(first_dot_epochs.info['ch_names'][ix + bidx])       
        dACC_names.append(pairs_name[ix])
        dACC_indices.append(ix)        
        
    # control roi
    if pairs_name[ix][0] in ROIs['insula'] or pairs_name[ix][1] in ROIs['insula']:
        insula_list.append(first_dot_epochs.info['ch_names'][ix + bidx])       
        insula_names.append(pairs_name[ix])
        insula_indices.append(ix)
    if pairs_name[ix][0] in ROIs['dlpfc'] or pairs_name[ix][1] in ROIs['dlpfc']:
        dlpfc_list.append(first_dot_epochs.info['ch_names'][ix + bidx])       
        dlpfc_names.append(pairs_name[ix])
        dlpfc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['sfg'] or pairs_name[ix][1] in ROIs['sfg']:
        sfg_list.append(first_dot_epochs.info['ch_names'][ix + bidx])       
        sfg_names.append(pairs_name[ix])
        sfg_indices.append(ix)   
    if pairs_name[ix][0] in ROIs['mfg'] or pairs_name[ix][1] in ROIs['mfg']:
        mfg_list.append(first_dot_epochs.info['ch_names'][ix + bidx])       
        mfg_names.append(pairs_name[ix])
        mfg_indices.append(ix)     
    if pairs_name[ix][0] in ROIs['ec'] or pairs_name[ix][1] in ROIs['ec']:
        ec_list.append(first_dot_epochs.info['ch_names'][ix + bidx])       
        ec_names.append(pairs_name[ix])
        ec_indices.append(ix)        
        

In [ ]:
# Print and check frequencies
print(freqs)
print(n_cycles)
print(time_bin)
print(band_width)


# Main Regions of Interest

## Hippocampus

In [ ]:
hc_list

In [ ]:
# only ROI of interest
first_dot_roi = first_dot_epochs.copy().pick_channels(hc_list)


In [ ]:
# Resample to 1000 
if first_dot_roi.info['sfreq'] > 1000:
    first_dot_roi= first_dot_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(first_dot_roi,freqs,n_cycles)


In [ ]:
# Crop to time of interest

roi_tfr.crop(tmin = -1, tmax = 4)

In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/first_dot/hc-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,4), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average Hippocampal TFR at First Dot, n={len(hc_list)}", subject, 'hc_firstdot')


## Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'First Dot')
    


## Export freq data locked to turn around time to csvs

# OFC

In [ ]:
ofc_list

In [ ]:
# only roi of interest
first_dot_roi = first_dot_epochs.copy().pick_channels(ofc_list)


In [ ]:
# Resample to 1000 
if first_dot_roi.info['sfreq'] > 1000:
    first_dot_roi= first_dot_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(first_dot_roi,freqs,n_cycles)

In [ ]:
roi_tfr.crop(tmin = -1, tmax = 4)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/first_dot/ofc-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,4), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average OFC TFR at First Dot, n={len(ofc_list)}", subject, 'ofc_firstdot')


### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'First Dot')

# Amygdala

In [ ]:
amyg_list

In [ ]:
# only roi 
first_dot_roi = first_dot_epochs.copy().pick_channels(amyg_list)


In [ ]:
# Resample to 1000 
if first_dot_roi.info['sfreq'] > 1000:
    first_dot_roi= first_dot_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(first_dot_roi,freqs,n_cycles)


In [ ]:
roi_tfr.crop(tmin = -1, tmax = 4)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/first_dot/amyg-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,4), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average Amygdala TFR at First Dot, n={len(amyg_list)}", subject, 'amyg_firstdot')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'First Dot')

## Cingulate

In [ ]:
cing_list

In [ ]:
# only roi 
first_dot_roi = first_dot_epochs.copy().pick_channels(cing_list)


In [ ]:
# Resample to 1000 
if first_dot_roi.info['sfreq'] > 1000:
    first_dot_roi= first_dot_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(first_dot_roi,freqs,n_cycles)


In [ ]:
roi_tfr.crop(tmin = -1, tmax = 4)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/first_dot/cing-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,4), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average Ant. Cingulate TFR at First Dot, n={len(cing_list)}", subject, 'cing_firstdot')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'First Dot')

## sgACC

In [ ]:
sgACC_list

In [ ]:
# Plot TFR
tmp_tfr = roi_tfr.copy().pick_channels(sgACC_list)
plot_average_tfr(tmp_tfr, f"Average sgACC TFR at First Dot, n={len(sgACC_list)}", subject, 'sgacc_firstdot')

### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [15, 11]

for ch in sgACC_list:
    chix = cing_list.index(ch)
    plot_channel_tfr(roi_tfr, chix, ch, 'First Dot')
    

## dACC

In [ ]:
dACC_list

In [ ]:
# Plot TFR
tmp_tfr = roi_tfr.copy().pick_channels(dACC_list)
plot_average_tfr(tmp_tfr, f"Average dACC TFR at First Dot, n={len(dACC_list)}", subject, 'dacc_firstdot')

### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [15, 11]

for ch in dACC_list:
    chix = cing_list.index(ch)
    plot_channel_tfr(roi_tfr, chix, ch, 'First Dot')


# Control Regions

## Insula

In [ ]:
insula_list

In [ ]:
# Only ROI
first_dot_roi = first_dot_epochs.copy().pick_channels(insula_list)


In [ ]:
# Resample to 1000 
if first_dot_roi.info['sfreq'] > 1000:
    first_dot_roi= first_dot_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(first_dot_roi,freqs,n_cycles)


In [ ]:
roi_tfr.crop(tmin = -1, tmax = 4)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/first_dot/insula-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,4), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average Insula TFR at First Dot, n={len(insula_list)}", subject, 'insula_firstdot')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'First Dot')

## dlPFC

In [ ]:
dlpfc_list

In [ ]:
# Only ROI
first_dot_roi = first_dot_epochs.copy().pick_channels(dlpfc_list)


In [ ]:
# Resample to 1000 
if first_dot_roi.info['sfreq'] > 1000:
    first_dot_roi= first_dot_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(first_dot_roi,freqs,n_cycles)

In [ ]:
roi_tfr.crop(tmin = -1, tmax = 4)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/first_dot/dlpfc-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,4), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average dlPFC TFR at First Dot, n={len(dlpfc_list)}", subject, 'dlpfc_firstdot')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'First Dot')

## MFG

In [ ]:
mfg_list

In [ ]:
# Plot TFR
tmp_tfr = roi_tfr.copy().pick_channels(mfg_list)
plot_average_tfr(tmp_tfr, f"Average MFG TFR at First Dot, n={len(mfg_list)}", subject, 'mfg_firstdot')

### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [15, 11]

for ch in mfg_list:
    chix = dlpfc_list.index(ch)
    plot_channel_tfr(roi_tfr, chix, ch, 'First Dot')
    

## SFG

In [ ]:
sfg_list

In [ ]:
# Plot TFR
tmp_tfr = roi_tfr.copy().pick_channels(sfg_list)
plot_average_tfr(tmp_tfr, f"Average SFG TFR at First Dot, n={len(sfg_list)}", subject, 'sfg_firstdot')

### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [15, 11]

for ch in sfg_list:
    chix = dlpfc_list.index(ch)
    plot_channel_tfr(roi_tfr, chix, ch, 'First Dot')

## Entorhinal Cortex

In [ ]:
ec_list

In [ ]:
# Only ROI
first_dot_roi = first_dot_epochs.copy().pick_channels(ec_list)

In [ ]:
# Resample to 1000 
if first_dot_roi.info['sfreq'] > 1000:
    first_dot_roi= first_dot_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(first_dot_roi,freqs,n_cycles)

In [ ]:
roi_tfr.crop(tmin = -1, tmax = 4)

In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/first_dot/ec-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,4), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average Entorhinal TFR at First Dot, n={len(ec_list)}", subject, 'ec_firstdot')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'First Dot')